In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.image import imread
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.nn import Parameter
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [3]:
# path

trainset_path = '/dataset/casia_webface_imgs_112_112'
testset_path = '/dataset/LFW/lfw'

# training dataset preprocessing

trainset_preprocess = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# test dataset preprocessing

testset_preprocess = transforms.Compose([
    transforms.ToTensor()
])

dataset = ImageFolder(trainset_path, transform=trainset_preprocess)
#testset = ImageFolder(trainset_path, transform=testset_preprocess)

train_set, test_set = torch.utils.data.random_split(dataset, [440623, 50000])


In [4]:

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=32, shuffle=True, num_workers=2)


In [5]:
# dataset

DATASET_CLASS = len(dataset.classes)
DATASET_DATA = len(dataset)


# same size of Celeb-500K-2R 98.2% Acc on LFW, VR@FAR=0 57.67% (ResNet20)
print(DATASET_CLASS)
print(DATASET_DATA)


10572
490623


In [6]:
class Block(nn.Module):
    def __init__(self, in_planes, out_planes, expansion, stride):
        super(Block, self).__init__()
        self.stride = stride

        planes = expansion * in_planes
        # channel expansion
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        # depthwise convolution
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, groups=planes, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.conv3 = nn.Conv2d(planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_planes)

        self.shortcut = nn.Sequential()
        if stride == 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_planes),
                nn.ReLU(inplace=True),
            )

    def forward(self, x):
        out = F.relu6(self.bn1(self.conv1(x)))
        out = F.relu6(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out + self.shortcut(x) if self.stride==1 else out

        return out


class MobileNetV2(nn.Module):
    cfg = [(1,  16, 1, 1),
           (6,  24, 2, 1),  
           (6,  32, 3, 2),
           (6,  64, 4, 2),
           (6,  96, 3, 1),
           (6, 160, 3, 2),
           (6, 224, 1, 1)]

    def __init__(self, num_classes=10572):
        super(MobileNetV2, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32)
        self.conv2 = nn.Conv2d(224, 256, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(256)
        self.linear = nn.Linear(256, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for expansion, out_planes, num_blocks, stride in self.cfg:
            strides = [stride] + [1]*(num_blocks-1)
            for stride in strides:
                layers.append(Block(in_planes, out_planes, expansion, stride))
                in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.relu(self.bn2(self.conv2(out)))
        
        out = F.avg_pool2d(out, 14)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [7]:
train_losses = []
train_accuracy = [] 
test_losses = []
test_accuracy = []

def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, prediction = outputs.max(1)

        total += labels.size(0)
        correct += prediction.eq(labels).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current train accuracy:', str(prediction.eq(labels).sum().item() / labels.size(0)))
            print('Current train average loss:', loss.item() / labels.size(0))

            train_losses.append(loss.item() / labels.size(0))
            train_accuracy.append(prediction.eq(labels).sum().item() / labels.size(0))
            
    print('\nTrain accuarcy:', correct / total)
    print('Train average loss:', train_loss / total)

In [8]:
def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        total += labels.size(0)

        outputs = net(images)
        loss += criterion(outputs, labels).item()

        _, prediction = outputs.max(1)
        correct += prediction.eq(labels).sum().item()

    print('\nTest accuarcy:', correct / total)
    print('Test average loss:', loss / total)
    test_losses.append(loss / total)
    test_accuracy.append(correct / total)

    state = {
        'net': net.state_dict()
    }

    file_name = 'CASIA_MobileNet_v2.pt'
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved')

In [9]:
net = MobileNetV2()
net = net.to(device)

learning_rate = 0.1
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)

In [10]:
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    #if epoch >= 50:
    if epoch >= 50:
        lr /= 10
    #if epoch >= 100:
    if epoch >= 100:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [11]:
# count_parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [12]:
print(count_parameters(net)/1000000,str('M'))

4.625036 M


In [13]:
pip install update

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install ptflops

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install --upgrade git+https://github.com/sovrasov/flops-counter.pytorch.git

  Cloning https://github.com/sovrasov/flops-counter.pytorch.git to /tmp/pip-req-build-4upm6a2u
  Created wheel for ptflops: filename=ptflops-0.6.6-py3-none-any.whl size=9719 sha256=70684615aad282970090633bd3608dbd1bbb350f9184604e34dab08b9cfcc937
  Stored in directory: /tmp/pip-ephem-wheel-cache-jr02i11v/wheels/b7/08/f3/c24c594062d3fe3a282194cfd29bc4d05f7496f7e971b7645a
Successfully built ptflops
  Attempting uninstall: ptflops
    Found existing installation: ptflops 0.6.6
    Uninstalling ptflops-0.6.6:
      Successfully uninstalled ptflops-0.6.6
Note: you may need to restart the kernel to use updated packages.


In [16]:
# Multiply-Accumulate

import torchvision.models as models
import torch
from ptflops import get_model_complexity_info

with torch.cuda.device(0):
    macs, params = get_model_complexity_info(net, (3, 112, 112), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
    print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
    print('{:<30}  {:<8}'.format('Number of parameters: ', params))
    print('= ')

MobileNetV2(
  4.625 M, 100.000% Params, 1.122 GMac, 100.000% MACs, 
  (conv1): Conv2d(0.001 M, 0.019% Params, 0.011 GMac, 0.966% MACs, 3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(0.0 M, 0.001% Params, 0.001 GMac, 0.072% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    1.849 M, 39.983% Params, 1.096 GMac, 97.709% MACs, 
    (0): Block(
      0.003 M, 0.055% Params, 0.032 GMac, 2.845% MACs, 
      (conv1): Conv2d(0.001 M, 0.022% Params, 0.013 GMac, 1.145% MACs, 32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(0.0 M, 0.001% Params, 0.001 GMac, 0.072% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(0.0 M, 0.006% Params, 0.004 GMac, 0.322% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(0.0 M, 0.001% Params, 0.001 GMac, 0.072% MACs, 32, eps=1e-05, mo

In [ ]:
start_time = time.time()

for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)
    print('\nTime elapsed:', time.time() - start_time)


[ Train epoch: 0 ]

Current batch: 0
Current train accuracy: 0.0
Current train average loss: 0.290746808052063

Current batch: 100
Current train accuracy: 0.0
Current train average loss: 0.2830881178379059

Current batch: 200
Current train accuracy: 0.0
Current train average loss: 0.2856045961380005

Current batch: 300
Current train accuracy: 0.0
Current train average loss: 0.273034006357193

Current batch: 400
Current train accuracy: 0.0
Current train average loss: 0.2785714864730835

Current batch: 500
Current train accuracy: 0.0
Current train average loss: 0.2811509370803833

Current batch: 600
Current train accuracy: 0.0
Current train average loss: 0.27899518609046936

Current batch: 700
Current train accuracy: 0.0
Current train average loss: 0.27694523334503174

Current batch: 800
Current train accuracy: 0.0
Current train average loss: 0.26692748069763184

Current batch: 900
Current train accuracy: 0.0
Current train average loss: 0.28205645084381104

Current batch: 1000
Current t

In [ ]:
df_train_l = pd.DataFrame(train_losses, columns = ['train_losses'])
df_train_l.plot(color = "#ff0000")
plt.plot(df_train_l, marker = '*', color = 'r')
plt.show()



In [ ]:
df_train_a = pd.DataFrame(train_accuracy, columns = ['train_accuracy'])
df_train_a.plot(color = "#ff1111")
plt.plot(df_train_a, marker = '.', color = 'b')
plt.show()


In [ ]:
df_test_l = pd.DataFrame(test_losses, columns = ['test_losses'])
df_test_l.plot(color = "#ff0000")
plt.plot(df_test_l, marker = '*', color = 'r')
plt.show()


In [ ]:
df_test_a = pd.DataFrame(test_accuracy, columns = ['test_accuracy'])
df_test_a.plot(color = "#ff0000")
plt.plot(df_test_a, marker = '*', color = 'r')
plt.show()
